In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("../../../XRaySimulation2")

from XRaySimulation import util

In [4]:
def get_bragg_reflectivity_with_phonon(kin_grid,
                                       d,
                                       h,
                                       n,
                                       q,
                                       omega,
                                       p,
                                       chi0,
                                       chih,
                                       chihbar):
    # ---------------------------------------------------
    # Get the output wave-vector
    # ---------------------------------------------------
    klen_grid = np.linalg.norm(kin_grid, axis=-1)

    # Define commonly used geometric quantities for the calculation
    gamma_0 = np.dot(kin_grid, n) / klen_grid
    gamma_h = np.dot(kin_grid + h[np.newaxis, :], n) / klen_grid
    b = gamma_0 / gamma_h

    alpha = (2 * np.dot(kin_grid, h) + np.sum(np.square(h))) / np.square(klen_grid)

    # Get surface momentrum transfer momentum tranfer 
    m_trans = np.multiply(klen_grid, -gamma_h - np.sqrt(gamma_h ** 2 - alpha))

    # Update the kout_grid
    kout_grid = (kin_grid + h[np.newaxis, :]
                 + np.multiply(m_trans[:, np.newaxis], n[np.newaxis, :]))

    # -----------------------------------------------------
    # Perfect crystal
    # ------------------------------------------------------
    y = 0.5 / np.sqrt(np.abs(b, dtype=np.complex128) * chih * chihbar)
    y *= b * alpha + chi0 * (1 - b)
    y1 = -y + np.sqrt(y ** 2 - 1)
    y2 = -y - np.sqrt(y ** 2 - 1)

    scriptG = np.sqrt(np.abs(b) * chih * chihbar) / chihbar
    scriptA = klen_grid * d * np.sqrt(chihbar * chih / gamma_0 / gamma_h)

    eta_d_1 = chi0 * klen_grid * d / 2. / gamma_0 + scriptA / 2. * y1
    eta_d_2 = chi0 * klen_grid * d / 2. / gamma_0 + scriptA / 2. * y2

    capR1 = scriptG * y1
    capR2 = scriptG * y2

    phase_term = np.exp(1.j * (eta_d_1 - eta_d_2))
    R0H = capR1 * capR2 * (1 - phase_term)
    R0H /= (capR2 - capR1 * phase_term)

    R00 = (capR2 - capR1) / (capR2 - capR1 * phase_term)
    R00 *= np.exp(1.j * eta_d_1)

    # -----------------------------------------------------
    # phonon contribution
    # ------------------------------------------------------
    delta_chih = 1.j * chih * np.dot(h, p)
    delta_chihbar = 1.j * chihbar * np.dot(h, p)
    beta_h = alpha - chi0 / 2.

    capA0 = 2. / klen_grid * np.dot(kin_grid / klen_grid[:, np.newaxis], q)
    capA0 -= 2 * omega / util.c / klen_grid

    capAh = 2. / klen_grid * np.dot((kin_grid + h[np.newaxis, :]) / klen_grid[:, np.newaxis], q)
    capAh -= 2 * omega / util.c / klen_grid

    d_plus = 0.5 * (chih * delta_chihbar * R0H + capA0 * delta_chih * R00)
    d_plus /= (capA0 * (capAh + 2 * beta_h) - chih * chihbar)

    d_minus = 0.5 * (chih * delta_chihbar * R0H - capA0 * delta_chih * R00)
    d_minus /= (-capA0 * (-capAh + 2 * beta_h) - chih * chihbar)

    return R0H, R00, d_plus, d_minus
    


In [5]:
# I feel confused about this implementation since it seems that I need to rock the crystal rather than the X-ray energy.
# therefore I cannot really test this algorithm
# instead, I need to get a new implementation that really change the crystal